In [1]:
import numpy as np
import pandas as pd
import random
from fastai.tabular.transform import add_cyclic_datepart
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [57]:
! ls

55175_105481_compressed_yoochoose-buys.dat.zip
55175_105481_compressed_yoochoose-clicks.dat.zip
55175_105481_compressed_yoochoose-test.dat.zip
preprocessing.ipynb
progress report.docx
yoochoose-buys.dat
yoochoose-clicks.dat
yoochoose-test.dat


In [26]:
path_buys = "yoochoose-buys.dat"
path_clicks = "yoochoose-clicks.dat"
path_test = "yoochoose-test.dat"

In [27]:
buys_head = ["session","time", "item", "price", "quantity"]
clicks_head = ["session","time", "item", "category"]
test_head = ["session","time", "item", "category"]

In [28]:
test = pd.read_csv(path_test, sep=',', names=test_head, parse_dates=['time'])
buys = pd.read_csv(path_buys, sep=',', names=buys_head, parse_dates=['time'])
buys = buys.sort_values(by=["time", "session"])
clicks = pd.read_csv(path_clicks, sep=',', names=clicks_head, parse_dates=[
                 'time'], converters={"category": lambda c: -1 if c == "S" else c})
clicks = clicks.sort_values(by=["time", "session"])

/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
uniq_clicks = clicks.session.unique()
uniq_buys = buys.session.unique()

In [30]:
# Make sets to give a decent sample of data.
c_not_b = set(uniq_clicks) - set(uniq_buys)
c_and_b = set(uniq_clicks).intersection(set(uniq_buys))

In [31]:
print(len(c_not_b), len(c_and_b))
assert len(c_not_b)+len(c_and_b) == len(uniq_clicks)

8740033 509696


In [57]:
num = 30000
cnb = random.sample(c_not_b, num)
cab = random.sample(c_and_b, num)

In [58]:
cl = clicks.loc[clicks.session.isin(cnb+cab)]
bu = buys.loc[buys.session.isin(cab)]
cl.shape, bu.shape

((297594, 4), (67421, 5))

In [59]:
cl.head()

,session,time,item,category
63316,18834,2014-04-01 03:47:51.468000+00:00,214819719,0
63317,18834,2014-04-01 03:49:17.031000+00:00,214718169,0
63318,18834,2014-04-01 03:49:46.058000+00:00,214832559,0
1366111,443348,2014-04-01 04:49:17.355000+00:00,214832559,0
1366112,443348,2014-04-01 04:50:50.792000+00:00,214832559,0


In [60]:
bu.head()

,session,time,item,price,quantity
14504,265497,2014-04-01 05:23:23.944000+00:00,214840483,1674,1
49269,432562,2014-04-01 05:51:49.336000+00:00,214829810,13509,1
49270,432562,2014-04-01 05:51:49.337000+00:00,214663976,2931,1
29391,383381,2014-04-01 05:52:49.720000+00:00,214697456,1151,3
41185,299597,2014-04-01 06:07:09.461000+00:00,214839313,4188,1


In [62]:
cl.loc[:, "event"] = 0
bu.loc[:, "event"] = 1

In [63]:
union=pd.concat([cl, bu], ignore_index=True, sort=False).sort_values(by=['session','time'])

In [64]:
union.head()

,session,time,item,category,event,price,quantity
13018,87,2014-04-07 06:19:08.914000+00:00,214652220,0,0,NaN,NaN
13020,87,2014-04-07 06:19:20.979000+00:00,214840483,0,0,NaN,NaN
13021,87,2014-04-07 06:19:28.762000+00:00,214840483,0,0,NaN,NaN
13045,87,2014-04-07 06:26:01.516000+00:00,214717286,0,0,NaN,NaN
13046,87,2014-04-07 06:26:15.176000+00:00,214558807,0,0,NaN,NaN


In [65]:
union.time.min()

Timestamp('2014-04-01 03:47:51.468000+0000', tz='UTC')

In [66]:
union.loc[:, "hour"] = union.time.dt.hour
union.loc[:, "month"] = union.time.dt.month
union.loc[:, "dow"] = union.time.dt.dayofweek

In [67]:
# union.category = union.category.fillna(-1)

In [68]:
union = union.drop(columns=["price","quantity","category"])

In [69]:
union

,session,time,item,event,hour,month,dow
13018,87,2014-04-07 06:19:08.914000+00:00,214652220,0,6,4,0
13020,87,2014-04-07 06:19:20.979000+00:00,214840483,0,6,4,0
13021,87,2014-04-07 06:19:28.762000+00:00,214840483,0,6,4,0
13045,87,2014-04-07 06:26:01.516000+00:00,214717286,0,6,4,0
13046,87,2014-04-07 06:26:15.176000+00:00,214558807,0,6,4,0
...,...,...,...,...,...,...,...
295448,11561884,2014-09-26 18:03:22.661000+00:00,214555104,0,18,9,4
295287,11562067,2014-09-26 16:11:22.515000+00:00,214850516,0,16,9,4
297345,11562118,2014-09-29 14:32:19.138000+00:00,214709702,0,14,9,0
297346,11562118,2014-09-29 14:32:37.129000+00:00,214709702,0,14,9,0


In [70]:
df = union.copy()
df.loc[:,'idx'] = df.groupby('session').cumcount()+1
# df = df.pivot_table(index=['session'], columns='idx', 
#                     values=['time', 'item', "hour", "month", 'dow'], aggfunc='first')

In [71]:
df.head()

,session,time,item,event,hour,month,dow,idx
13018,87,2014-04-07 06:19:08.914000+00:00,214652220,0,6,4,0,1
13020,87,2014-04-07 06:19:20.979000+00:00,214840483,0,6,4,0,2
13021,87,2014-04-07 06:19:28.762000+00:00,214840483,0,6,4,0,3
13045,87,2014-04-07 06:26:01.516000+00:00,214717286,0,6,4,0,4
13046,87,2014-04-07 06:26:15.176000+00:00,214558807,0,6,4,0,5


In [72]:
df.item.nunique()

18465

In [73]:
sessions = df.session.unique()

In [74]:
len(sessions)

60000

In [75]:
cool2 = df.groupby("session").shift(1)["time"].dt.tz_localize('UTC')
df.loc[:,"timediff"] = (df.time - cool2).dt.seconds
df.loc[:,"timediff"] = np.log2(df.timediff).replace([np.inf, -np.inf], np.nan).fillna(-1).astype(int)

/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [76]:
from tqdm import tqdm

In [78]:
def renew(df):
    df = df.pivot_table(index=['session'], columns=tmp.index, 
                        values=['timediff', 'item', "hour", "month", 'dow', 'event','time'], aggfunc='first')

    df = df.sort_index(axis=1, level=1)
    df.columns = [f'{x}_{y}' for x,y in df.columns]
    df = df.reset_index().fillna(0)
    return df.copy()

In [ ]:
final = pd.DataFrame()
res = []
for sid in tqdm(sessions):
    dd = df.loc[df.session == sid]
    n = dd.idx.max()
    for c in range(2, n):
        mini_dd = dd.loc[(dd.idx <= c) & (dd.idx > c-20)]
        g = mini_dd.shape[0]
        lst = list(range(20-g+1, 21))
        mini_dd.index = lst
        tmp = mini_dd.reindex(mini_dd.index.tolist() + list(range(1, 21-g)))
        tmp.loc[:,"session"] = mini_dd.session.max()
        tmp = tmp.fillna(-1)
        row = renew(tmp)
        res.append(row.values[0])

  1%|          | 377/60000 [00:34<1:20:28, 12.35it/s]

In [350]:
finals = pd.DataFrame(res, columns=row.columns)
finals.to_csv()

In [361]:
nn = len(res)

In [364]:
df = res[0]
for i in tqdm(range(10000)):
    
    df = df.append(res[i])

100%|██████████| 10000/10000 [04:54<00:00, 33.91it/s]


In [365]:
df.to_csv("sampling")

In [381]:
val = [f.values[0] for f in res]

In [384]:
gg = pd.DataFrame(val)

In [386]:
gg.columns = res[0].columns

In [388]:
gg.to_csv("final")

In [401]:
go = gg.loc[gg.session==189]

In [424]:
167914 * 0.7

117539.79999999999

In [493]:
df = union.copy()
df.head(3)

,event,item,session,time,hour,month,dow
4324,0,214613865,189,2014-04-04 07:14:31.423000+00:00,7,4,4
4326,0,214613865,189,2014-04-04 07:14:45.348000+00:00,7,4,4
4329,0,214613870,189,2014-04-04 07:16:52.475000+00:00,7,4,4


In [20]:
mini_dd

NameError: name 'mini_dd' is not defined

In [22]:
sessions

NameError: name 'sessions' is not defined

100%|██████████| 40000/40000 [30:32<00:00, 21.82it/s]  


In [613]:
finals.head()

,session,dow_1,event_1,hour_1,item_1,month_1,timediff_1,dow_2,event_2,hour_2,...,hour_19,item_19,month_19,timediff_19,dow_20,event_20,hour_20,item_20,month_20,timediff_20
0,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,7.0,214613865.0,4.0,-1.0,4.0,0.0,7.0,214613865.0,4.0,3.0
1,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,7.0,214613865.0,4.0,3.0,4.0,0.0,7.0,214613870.0,4.0,6.0
2,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214821022.0,4.0,-1.0,1.0,0.0,11.0,214718169.0,4.0,4.0
3,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214718169.0,4.0,4.0,1.0,0.0,11.0,214821022.0,4.0,5.0
4,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214821022.0,4.0,5.0,1.0,0.0,11.0,214718169.0,4.0,5.0


In [614]:
finals.to_csv("completed.csv")

In [596]:
es = df.groupby("session").count().reset_index()
es.loc[es.event == 108]

,session,event,item,time,hour,month,dow,timediff,idx
31728,9152034,108,108,108,108,108,108,108,108


In [560]:
res

[array([ 1.890000e+02, -1.000000e+00, -1.000000e+00, -1.000000e+00, ...,  7.000000e+00,  2.146139e+08,  4.000000e+00,
        -1.000000e+00]),
 array([ 1.890000e+02, -1.000000e+00, -1.000000e+00, -1.000000e+00, ...,  7.000000e+00,  2.146139e+08,  4.000000e+00,
         3.000000e+00])]

In [572]:
tester = gg.head(1)
tester.values[0]

array([189, -1.0, -1.0, -1.0, ..., 4.0, Timestamp('2014-04-04 07:14:31.423000+0000', tz='UTC'), 214613865.0, 0.0],
      dtype=object)

In [577]:
tester

,session,dow_1,hour_1,item_1,month_1,time_1,dow_2,hour_2,item_2,month_2,...,item_19,month_19,time_19,dow_20,hour_20,item_20,month_20,time_20,y,y_event
0,189,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1,4.0,7.0,214613865.0,4.0,2014-04-04 07:14:31.423000+00:00,214613865.0,0.0


In [578]:
tester.values[0][1:101].reshape(20,5)

array([[-1.0, -1.0, -1.0, -1.0, -1],
       [-1.0, -1.0, -1.0, -1.0, -1],
       [-1.0, -1.0, -1.0, -1.0, -1],
       [-1.0, -1.0, -1.0, -1.0, -1],
       ...,
       [-1.0, -1.0, -1.0, -1.0, -1],
       [-1.0, -1.0, -1.0, -1.0, -1],
       [-1.0, -1.0, -1.0, -1.0, -1],
       [4.0, 7.0, 214613865.0, 4.0, Timestamp('2014-04-04 07:14:31.423000+0000', tz='UTC')]], dtype=object)

In [3]:
gg = pd.read_csv("completed.csv")

In [5]:
gg

,Unnamed: 0,session,dow_1,event_1,hour_1,item_1,month_1,timediff_1,dow_2,event_2,...,hour_19,item_19,month_19,timediff_19,dow_20,event_20,hour_20,item_20,month_20,timediff_20
0,0,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,7.0,214613865.0,4.0,-1.0,4.0,0.0,7.0,214613865.0,4.0,3.0
1,1,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,7.0,214613865.0,4.0,3.0,4.0,0.0,7.0,214613870.0,4.0,6.0
2,2,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214821022.0,4.0,-1.0,1.0,0.0,11.0,214718169.0,4.0,4.0
3,3,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214718169.0,4.0,4.0,1.0,0.0,11.0,214821022.0,4.0,5.0
4,4,439.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,11.0,214821022.0,4.0,5.0,1.0,0.0,11.0,214718169.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167909,167909,11560733.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,18.0,214854731.0,9.0,3.0,2.0,0.0,18.0,214848658.0,9.0,6.0
167910,167910,11560733.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,18.0,214848658.0,9.0,6.0,2.0,0.0,18.0,214854347.0,9.0,6.0
167911,167911,11560733.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,18.0,214854347.0,9.0,6.0,2.0,1.0,18.0,214848658.0,9.0,6.0
167912,167912,11560733.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,18.0,214848658.0,9.0,6.0,2.0,1.0,18.0,214855209.0,9.0,-1.0


In [12]:
ggtime = gg.sort_values(by="month_20", ascending=True)
ggtime.loc[ggtime.month_20<=7].to_csv("TRAIN_1")
ggtime.loc[ggtime.month_20==8].to_csv("VALID_1")
ggtime.loc[ggtime.month_20==9].to_csv("TEST_1")

In [15]:
ggtime.loc[ggtime.month_20==9].shape[0]

29236

In [16]:
ggtime.shape

(167914, 122)

In [17]:
29236/167914

0.1741129387662732

In [347]:
final.to_csv("final")

In [335]:
mini_dd

,event,item,session,time,hour,month,dow,idx
19,0,214613865,189,2014-04-04 07:14:31.423000+00:00,7,4,4,1
20,0,214613865,189,2014-04-04 07:14:45.348000+00:00,7,4,4,2


In [293]:
c

2

In [13]:
ggtime

,Unnamed: 0,session,dow_1,event_1,hour_1,item_1,month_1,timediff_1,dow_2,event_2,...,hour_19,item_19,month_19,timediff_19,dow_20,event_20,hour_20,item_20,month_20,timediff_20
0,0,189.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,7.0,214613865.0,4.0,-1.0,4.0,0.0,7.0,214613865.0,4.0,3.0
20314,20314,1241224.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,15.0,214821285.0,4.0,7.0,2.0,0.0,15.0,214584907.0,4.0,5.0
20313,20313,1241224.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,15.0,214535621.0,4.0,7.0,2.0,0.0,15.0,214821285.0,4.0,7.0
20312,20312,1241224.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,15.0,214535621.0,4.0,-1.0,2.0,0.0,15.0,214535621.0,4.0,7.0
20311,20311,1241138.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,6.0,214829747.0,4.0,-1.0,6.0,0.0,7.0,214828991.0,4.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148416,148416,10263408.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,19.0,214843689.0,9.0,1.0,1.0,0.0,19.0,214854609.0,9.0,6.0
148415,148415,10263408.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,19.0,214843689.0,9.0,-1.0,1.0,0.0,19.0,214843689.0,9.0,1.0
148414,148414,10263238.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,19.0,214854352.0,9.0,6.0,0.0,1.0,19.0,214853113.0,9.0,-1.0
148412,148412,10263238.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,19.0,214718220.0,9.0,6.0,0.0,0.0,19.0,214854542.0,9.0,6.0


In [309]:
mini_dd

,event,item,session,time,hour,month,dow,idx
20,0,214613865,189,2014-04-04 07:14:31.423000+00:00,7,4,4,1


,session,item_1,time_1,item_2,time_2,item_3,time_3,item_4,time_4,item_5,...,item_16,time_16,item_17,time_17,item_18,time_18,item_19,time_19,item_20,time_20
0,189,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,...,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,-1,214613865.0,2014-04-04 07:14:31.423000+00:00


In [286]:
parse_dates=['time']

In [435]:
pd.read_csv?

In [288]:
renew(tmp)

,session,item_0,time_0,item_1,time_1,item_2,time_2,item_3,time_3,item_4,...,item_15,time_15,item_16,time_16,item_17,time_17,item_18,time_18,item_4324,time_4324
0,189,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,...,-1.0,-1,-1.0,-1,-1.0,-1,-1.0,-1,214613865.0,2014-04-04 07:14:31.423000+00:00


In [233]:
new = pd.DataFrame(columns = ["session","y"]+[f'{x}_{y}' for x in ['time', 'item', "hour", "month", 'dow', 'event'] for y in range(1,21)])

In [226]:
for ii in range(2, df.idx.max()+1):
    dd = df.loc[df.idx==ii]
    break

In [237]:
len(df.session.unique())

40000

In [238]:
new.session = df.session.unique()

In [239]:
new

,session,y,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,...,event_11,event_12,event_13,event_14,event_15,event_16,event_17,event_18,event_19,event_20
0,189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,11560368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39996,11560541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39997,11560733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39998,11560959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
df = union.copy()
df.loc[:,'idx'] = df.groupby('session').cumcount()+1
df = df.pivot_table(index=['session'], columns='idx', 
                    values=['time', 'item', "category"], aggfunc='first')

df = df.sort_index(axis=1, level=1)
df.columns = [f'{x}_{y}' for x,y in df.columns]
df = df.reset_index().fillna(0)

In [175]:
rhs = bu.groupby("session").agg({"item":list, "quantity":list}).copy()

In [188]:
df.loc[df.item_2!=0]

,session,category_1,item_1,time_1,category_2,item_2,time_2,category_3,item_3,time_3,...,time_98,category_99,item_99,time_99,category_100,item_100,time_100,category_101,item_101,time_101
0,189,0,214613865.0,2014-04-04 07:14:31.423000+00:00,0,214613865.0,2014-04-04 07:14:45.348000+00:00,0,214613870.0,2014-04-04 07:16:52.475000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
2,439,0,214821022.0,2014-04-01 11:07:47.597000+00:00,0,214718169.0,2014-04-01 11:08:12.854000+00:00,0,214821022.0,2014-04-01 11:08:45.914000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
3,589,0,214821024.0,2014-04-01 18:22:13.682000+00:00,0,214819427.0,2014-04-01 18:26:43.493000+00:00,0,214668249.0,2014-04-01 18:27:45.312000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
4,592,0,214716984.0,2014-04-02 13:50:46.026000+00:00,0,214832657.0,2014-04-02 13:52:31.358000+00:00,0,214716982.0,2014-04-02 13:52:59.137000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
5,669,0,214551917.0,2014-04-02 12:21:58.126000+00:00,0,214551917.0,2014-04-02 12:22:01.248000+00:00,0,214551917.0,2014-04-02 12:22:49.080000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,11560368,-1,214714755.0,2014-09-24 20:15:19.139000+00:00,-1,214854838.0,2014-09-24 20:19:04.500000+00:00,-1,214854332.0,2014-09-24 20:21:07.928000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39996,11560541,9,214837194.0,2014-09-27 21:52:22.230000+00:00,-1,214854217.0,2014-09-27 21:52:27.836000+00:00,9,214854802.0,2014-09-27 21:57:35.304000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39997,11560733,-1,214855209.0,2014-09-24 18:04:27.128000+00:00,-1,214855209.0,2014-09-24 18:05:13.023000+00:00,-1,214854332.0,2014-09-24 18:06:20.988000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39998,11560959,-1,214854731.0,2014-09-25 06:47:45.621000+00:00,-1,214853102.0,2014-09-25 06:48:24.579000+00:00,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0


In [185]:
cl

,session,time,item,category
1101459,328342,2014-04-01 03:30:29.541000+00:00,214821011,0
1101460,328342,2014-04-01 03:31:22.247000+00:00,214718160,0
492238,147492,2014-04-01 04:23:37.629000+00:00,214585633,0
492239,147492,2014-04-01 04:23:59.504000+00:00,214585633,0
492240,147492,2014-04-01 04:26:38.093000+00:00,214548834,0
...,...,...,...,...
32824618,11369729,2014-09-30 02:06:41.677000+00:00,214509647,2
32824619,11369729,2014-09-30 02:07:06.303000+00:00,214509776,2
32824620,11369729,2014-09-30 02:09:13.422000+00:00,214509776,2
32824621,11369729,2014-09-30 02:12:59.634000+00:00,214509695,2


,session,category_1,item_1,time_1,category_2,item_2,time_2,category_3,item_3,time_3,...,time_98,category_99,item_99,time_99,category_100,item_100,time_100,category_101,item_101,time_101
0,189,0,214613865.0,2014-04-04 07:14:31.423000+00:00,0,214613865.0,2014-04-04 07:14:45.348000+00:00,0,214613870.0,2014-04-04 07:16:52.475000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
1,341,0,214820441.0,2014-04-06 19:27:23.089000+00:00,0,0.0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0
2,439,0,214821022.0,2014-04-01 11:07:47.597000+00:00,0,214718169.0,2014-04-01 11:08:12.854000+00:00,0,214821022.0,2014-04-01 11:08:45.914000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
3,589,0,214821024.0,2014-04-01 18:22:13.682000+00:00,0,214819427.0,2014-04-01 18:26:43.493000+00:00,0,214668249.0,2014-04-01 18:27:45.312000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
4,592,0,214716984.0,2014-04-02 13:50:46.026000+00:00,0,214832657.0,2014-04-02 13:52:31.358000+00:00,0,214716982.0,2014-04-02 13:52:59.137000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,11560368,-1,214714755.0,2014-09-24 20:15:19.139000+00:00,-1,214854838.0,2014-09-24 20:19:04.500000+00:00,-1,214854332.0,2014-09-24 20:21:07.928000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39996,11560541,9,214837194.0,2014-09-27 21:52:22.230000+00:00,-1,214854217.0,2014-09-27 21:52:27.836000+00:00,9,214854802.0,2014-09-27 21:57:35.304000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39997,11560733,-1,214855209.0,2014-09-24 18:04:27.128000+00:00,-1,214855209.0,2014-09-24 18:05:13.023000+00:00,-1,214854332.0,2014-09-24 18:06:20.988000+00:00,...,0,0,0.0,0,0,0.0,0,0,0.0,0
39998,11560959,-1,214854731.0,2014-09-25 06:47:45.621000+00:00,-1,214853102.0,2014-09-25 06:48:24.579000+00:00,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0,0.0,0


In [178]:
bu[["item", "quantity"]]

,item,quantity
48350,214832557,1
48351,214840415,1
48352,214642059,1
11763,214820231,6
11764,214820231,6
...,...,...
1143191,214847724,1
1143192,214855012,1
1143193,214857568,2
1143194,214855012,1


In [153]:
# %debug
x, y = prep(cl, bu)

Prepping data for classification
Processing buys
Buys from 2014-04-01 03:30:29.541000+00:00 to 2014-09-30 02:43:01.566000+00:00
Buys grouped by session 40000 2
Clicks shape 45805 5
Processing clicks
Processed clicks shape 45805 8
Preprocessing - Grouping clicks into sessions


KeyError: 'Column not found: category'

In [116]:
processed_clicks = process_clicks(cl)
a, b = process_sessions(cl, limit=None)

Processing clicks


/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/darenma/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

Processed clicks shape 196890 7
Preprocessing - Grouping clicks into sessions


In [17]:
def make_training_set(clicks, buys):
    
    
    print(clicks.shape, buys.shape)
    df = pd.merge(clicks, buys, on="session_id", how="left", suffixes=["_C", "_B"])
    print(df.shape)
    return df

In [18]:
df = make_training_set(clicks, buys)

(33003944, 4) (1150753, 5)
(40596053, 8)


In [27]:
# Long to Wide

df.loc[:,'idx'] = df.groupby('session_id').cumcount()+1
df = df.pivot_table(index=['session_id'], columns='idx', 
                    values=['time', 'item_id', "category"], aggfunc='first')

df = df.sort_index(axis=1, level=1)
df.columns = [f'{x}_{y}' for x,y in df.columns]
df = df.reset_index()

In [38]:
agg_dict = {"item_id":list, "time":["min", "max", list], "category":[pd.Series.mode, list]}

In [51]:
cl = clicks.groupby("session_id").agg(agg_dict)

KeyboardInterrupt: 

In [183]:
cl

,session,time,item,category
1101459,328342,2014-04-01 03:30:29.541000+00:00,214821011,0
1101460,328342,2014-04-01 03:31:22.247000+00:00,214718160,0
492238,147492,2014-04-01 04:23:37.629000+00:00,214585633,0
492239,147492,2014-04-01 04:23:59.504000+00:00,214585633,0
492240,147492,2014-04-01 04:26:38.093000+00:00,214548834,0
...,...,...,...,...
32824618,11369729,2014-09-30 02:06:41.677000+00:00,214509647,2
32824619,11369729,2014-09-30 02:07:06.303000+00:00,214509776,2
32824620,11369729,2014-09-30 02:09:13.422000+00:00,214509776,2
32824621,11369729,2014-09-30 02:12:59.634000+00:00,214509695,2


In [184]:
bu

,session,time,item,price,quantity
48350,124332,2014-04-01 04:32:26.691000+00:00,214832557,2092,1
48351,124332,2014-04-01 04:32:26.692000+00:00,214840415,2092,1
48352,124332,2014-04-01 04:32:26.707000+00:00,214642059,5235,1
11763,13506,2014-04-01 04:47:38.458000+00:00,214820231,1046,6
11764,13506,2014-04-01 04:54:23.126000+00:00,214820231,1046,6
...,...,...,...,...,...
1143191,11500176,2014-09-30 02:09:28.909000+00:00,214847724,1046,1
1143192,11500176,2014-09-30 02:09:28.915000+00:00,214855012,313,1
1143193,11500176,2014-09-30 02:09:29.258000+00:00,214857568,1046,2
1143194,11500176,2014-09-30 02:09:29.362000+00:00,214855012,313,1


In [182]:
cl.session

,session,time,item,category
1101459,328342,2014-04-01 03:30:29.541000+00:00,214821011,0
1101460,328342,2014-04-01 03:31:22.247000+00:00,214718160,0
492238,147492,2014-04-01 04:23:37.629000+00:00,214585633,0
492239,147492,2014-04-01 04:23:59.504000+00:00,214585633,0
492240,147492,2014-04-01 04:26:38.093000+00:00,214548834,0
...,...,...,...,...
32824618,11369729,2014-09-30 02:06:41.677000+00:00,214509647,2
32824619,11369729,2014-09-30 02:07:06.303000+00:00,214509776,2
32824620,11369729,2014-09-30 02:09:13.422000+00:00,214509776,2
32824621,11369729,2014-09-30 02:12:59.634000+00:00,214509695,2


In [ ]:
def read_buys(limit=None):
    buys = pd.read_csv(f"{PATH}/yoochoose-buys.dat",
                    names=["session", "timestamp", "item", "price", "qty"],
                   parse_dates=["timestamp"])
    buys = buys.sort_values(by=["timestamp", "session"])
    if limit:
        buys = buys.iloc[:limit]
    return buys


def read_clicks(limit=None):
    print("Loading clicks")
    filename = f"{PATH}/yoochoose-clicks.dat"
    df = pd.read_csv(filename,
                     names=["session", "timestamp", "item", "category"],
                     parse_dates=["timestamp"],
                     converters={"category": lambda c: -1 if c == "S" else c})
    df = df.sort_values(by=["timestamp", "session"])
    if limit:
        df = df.iloc[:limit]
    print("Clicks shape %s %s" % df.shape)
    return df


def process_clicks(clicks, rolling_days=15):
    # Compute dwell time for each click
    print("Processing clicks")
    clicks['prev_ts'] = clicks.groupby('session')['timestamp'].transform(lambda x: x.shift())
    clicks['diff_prev'] = clicks["timestamp"] - clicks["prev_ts"] # in minutes
    clicks["dwell"] = clicks.groupby('session')['diff_prev'].transform(lambda x: x.shift(-1)).dt.seconds/60
    clicks = clicks.sort_values(by=["session", "timestamp"])
    print("Processed clicks shape %s %s" % clicks.shape)
    return clicks


def process_buys(limit=None):
    # Group into sessions, compute nr of items bought and set label column
    buys = read_buys(limit=limit)
    print("Processing buys")
    print("Buys from %s to %s" % (buys.timestamp.min(), buys.timestamp.max()))
    grouped = buys.groupby("session")
    buys_g = pd.DataFrame(index=grouped.groups.keys())
    buys_g["items_bought"] = grouped.item.count() # quantity may be zero which is weird so dont use it
    buys_g["is_buy"] = 1 # for easier merge later on
    buys_g.index.name = "session"
    print("Buys grouped by session %s %s" % buys_g.shape)
    return buys_g


def get_items_cats_percent(clicks, limit=None):
    buys = read_buys(limit=limit)
    # percent bought
    item_id_bought_pct = buys.item.value_counts(normalize=True)
    cat_id_viewed_pct = clicks.category.value_counts(normalize=True)
    item_id_viewed_pct = clicks.item.value_counts(normalize=True)

    return dict(views=dict(item=item_id_viewed_pct, cat=cat_id_viewed_pct), buys=item_id_bought_pct)


def process_sessions(processed_clicks, limit=None):
    print("Preprocessing - Grouping clicks into sessions")
    clicks = processed_clicks
    
    # Group clicks by session
    grouped = clicks.groupby("session")
    sessions = pd.DataFrame(index=grouped.groups.keys())
    
    # Session counters
    sessions["total_clicks"] = grouped.item.count()
    sessions["total_items"] = grouped.item.unique().apply(lambda x: len(x))
    sessions["total_cats"] = grouped.category.unique().apply(lambda x: len(x))
    print("Computed counters")
    
    # Session duration
    sessions["max_dwell"] = grouped.dwell.max()
    sessions["mean_dwell"] = grouped.dwell.mean()
    sessions["start_ts"] = grouped.timestamp.min()
    sessions["end_ts"] = grouped.timestamp.max()
    sessions["total_duration"] = (sessions["end_ts"] - sessions["start_ts"]).dt.seconds / 60
    print("Computed dwell and duration")
    
    # Click rate
    sessions["total_duration_secs"] = (sessions["end_ts"] - sessions["start_ts"]).dt.seconds
    sessions["click_rate"] = sessions["total_clicks"] / sessions["total_duration_secs"]
    sessions.click_rate = sessions.click_rate.replace(np.inf, np.nan)
    sessions.click_rate = sessions.click_rate.fillna(0)
    del sessions["total_duration_secs"]
    print("Computed click rate")
    
    # Dates
    #sessions = add_datepart(sessions, "start_ts", drop=False)
    #sessions = add_datepart(sessions, "end_ts", drop=False)
    sessions = add_cyclic_datepart(sessions, "start_ts", drop=False)
    sessions = add_cyclic_datepart(sessions, "end_ts", drop=False)
    print("Computed cyclic date parts")
    
    # What is the item and cat most viewed in this session?
    # How many times were they viewed?
    sessions["cat_most_viewed_n_times"] = grouped.category.value_counts().unstack().max(axis=1)
    sessions["cat_most_viewed"] = grouped.category.value_counts().unstack().idxmax(axis=1)
    sessions["item_most_viewed_n_times"] = grouped.item.value_counts().unstack().max(axis=1)
    sessions["item_most_viewed"] = grouped.item.value_counts().unstack().idxmax(axis=1)
    print("Computed most viewed item/cat per session")

    # For the item most viewed in each session, what is its global buy/view frequency?
    freqs = get_items_cats_percent(clicks, limit=limit)
    cat_views = pd.DataFrame(freqs["views"]["cat"])
    cat_views.columns = ["cat_views_freqs"]
    sessions = sessions.merge(cat_views, how="left", left_on="cat_most_viewed", right_index=True)
    sessions.cat_views_freqs = sessions.cat_views_freqs.fillna(0)
    item_views = pd.DataFrame(freqs["views"]["item"])
    item_views.columns = ["item_views_freqs"]
    sessions = sessions.merge(item_views, how="left", left_on="item_most_viewed", right_index=True)
    sessions.item_views_freqs = sessions.item_views_freqs.fillna(0)
    item_buys = pd.DataFrame(freqs["buys"])
    item_buys.columns = ["item_buys_freqs"]
    sessions = sessions.merge(item_buys, how="left", left_on="item_most_viewed", right_index=True)
    sessions.item_buys_freqs = sessions.item_buys_freqs.fillna(0)
    print("Computed most viewed/bought freqs")
    
    # Sorting sessions
    sessions = sessions.sort_values(by=["start_ts"])
    sessions.index.name = "session"
    
    print("Sessions shape %s %s" % sessions.shape)
    print("Sessions columns %s " % sessions.columns)
    print("Sessions from %s to %s" % (sessions.start_ts.min(), sessions.start_ts.max()))
    return sessions


def prep(limit=None):
    print("Prepping data for classification")
    buys = process_buys(limit=limit)
    clicks = read_clicks(limit=limit)
    processed_clicks = process_clicks(clicks)
    sessions = process_sessions(clicks, limit=limit)
    
    print("Merging clicks and buys")
    X = pd.merge(sessions, buys, how="left", left_index=True, right_index=True)
    
    X = X.sort_values(by=["start_ts"])
    y = X["is_buy"]
    y = y.fillna(0)
    
    X["cat_most_viewed"] = X["cat_most_viewed"].astype("float64")
    
    # Delete label
    del X["is_buy"]
    
    # Delete time columns (cant be used as is and we already have the cyclic date parts)
    del X["start_ts"]
    del X["end_ts"]
    
    return X, y

